# Three-particle potentials

In this notebook we'll explore the Stillinger-Weber three-particle potential applied to a system of silicon (Si) atoms.
The Stillinger-Weber potential is given by a two-particle potential similar to Lennard-Jones and a three-particle potential.
\begin{align}
    U(r)
    = \sum_{ij} U(r_{ij})
    + \sum_{ijk} U(r_{ij}, r_{ik}, \theta_{ijk}).
\end{align}
The two-particle potential is given by
\begin{align}
    U(r_{ij})
    = A \epsilon\left[
        B \left(
            \frac{\sigma}{r_{ij}}
        \right)^{p}
        - \left(
            \frac{\sigma}{r_{ij}}
        \right)^{q}
    \right]
    \exp\left(
        \frac{\sigma}{r_{ij} - a\sigma}
    \right),
\end{align}
where $A$, $B$, $\epsilon$, $\sigma$, $p$, $q$ and $a$ are coefficients that can be adjusted depending on the system we are perusing.
The three-particle potential has the form
\begin{align}
    U(r_{ij}, r_{ik}, \theta_{ijk})
    = \lambda \epsilon \left[
        \cos(\theta_{ijk})
        - \cos(\theta_{0})
    \right]^2
    \exp\left(
        \frac{\gamma\sigma}{r_{ij} - a\sigma}
    \right)
    \exp\left(
        \frac{\gamma\sigma}{r_{ik} - a\sigma}
    \right),
\end{align}
where we've introduced the new parameters $\lambda$ and $\gamma$.
Note that we've simplified the potentials as we've removed the indices that can be associated with every coefficient to specify different types of interactions between atoms.
Below we use the given parameter file.

In [2]:
%%writefile scripts/Si.sw
# DATE: 2007-06-11 CONTRIBUTOR: Aidan Thompson, athomps@sandia.gov
# CITATION: Stillinger and Weber,  Phys Rev B, 31, 5262, (1985)
# Stillinger-Weber parameters for various elements and mixtures
# multiple entries can be added to this file,
# LAMMPS reads the ones it needs
# these entries are in LAMMPS "metal" units:
#   epsilon = eV; sigma = Angstroms
#   other quantities are unitless
# format of a single entry (one or more lines):
#   element 1, element 2, element 3,
#   epsilon, sigma, a, lambda, gamma, costheta0, A, B, p, q, tol
# Here are the original parameters in metal units, for Silicon from:
#
# Stillinger and Weber,  Phys. Rev. B, v. 31, p. 5262, (1985)
#

Si # element 1
Si # element 2
Si # element 3
2.1683 # epsilon
2.0951 # sigma
1.80 # a
21.0 # lambda
1.20 # gamma
-0.333333333333 # cos(theta_0)
7.049556277 # A
0.6022245584 # B
4.0 # p
0.0 # q
0.0 # tol

Overwriting scripts/Si.sw


In the Lennard-Jones-scripts we only specify $\epsilon$, $\sigma$ and $r_c$ (cut-off).
Furthermore, we see that for $A = 4$, $B = 1$, $p = 12$, and $q = 6$ we almost recover the Lennard-Jones potential in the two-particle potential if we remove the exponential cut-off function.
With $p = 4$ we lower the repulsive effect of the two-particle potential and by setting $q = 0$ we use a fixed attractive force between the particles.

In [23]:
%%writefile scripts/si.in

units           metal
atom_style      atomic
atom_modify     map array
boundary        p p p
atom_modify     sort 0 0.0

# temperature
variable myTemp equal 1200.0

# diamond unit cell
variable myL equal 4
variable myscale equal 1.3

variable a equal 5.431*${myscale}
lattice         custom $a               &
                a1 1.0 0.0 0.0          &
                a2 0.0 1.0 0.0          &
                a3 0.0 0.0 1.0          &
                basis 0.0 0.0 0.0       &
                basis 0.0 0.5 0.5       &
                basis 0.5 0.0 0.5       &
                basis 0.5 0.5 0.0       &
                basis 0.25 0.25 0.25    &
                basis 0.25 0.75 0.75    &
                basis 0.75 0.25 0.75    &
                basis 0.75 0.75 0.25

region          myreg block     0 ${myL} &
                                0 ${myL} &
                                0 ${myL}

create_box      1 myreg
create_atoms    1 region myreg

mass            1       28.06

group Si type 1

velocity all create ${myTemp} 5287286 mom yes rot yes dist gaussian

pair_style      sw
pair_coeff * * scripts/Si.sw Si

neighbor        1.0 bin
neigh_modify    every 1 delay 10 check yes

timestep        1.0e-3
#fix             1 all nve
fix             1 all nvt temp ${myTemp} ${myTemp} 0.05

# Run simulation
thermo          10
dump 1 all custom 10 dat/si.lammpstrj id type x y z vx vy vz
run             10000

Overwriting scripts/si.in


In [24]:
!export OMP_NUM_THREADS=4 && \
    mpirun -np 4 lmp -in scripts/si.in

LAMMPS (4 Jan 2019)
  using 4 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 7.0603 7.0603 7.0603
Created orthogonal box = (0 0 0) to (28.2412 28.2412 28.2412)
  1 by 2 by 2 MPI processor grid
Created 512 atoms
  Time spent = 0.000469814 secs
512 atoms in group Si
Reading potential file scripts/Si.sw with DATE: 2007-06-11
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 4.77118
  ghost atom cutoff = 4.77118
  binsize = 2.38559, bins = 12 12 12
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair sw, perpetual
      attributes: full, newton on
      pair build: full/bin/atomonly
      stencil: full/bin/3d
      bin: standard
Setting up Verlet run ...
  Unit style    : metal
  Current step  : 0
  Time step     : 0.001
Per MPI rank memory allocation (min/avg/max) = 6.851 | 6.851 | 6.851 Mbytes
Step Temp E_pair E_mol TotEng Press 
       0         1200   -721.59958  

    1060    1279.7253   -1627.9912            0   -1543.4628   -15923.505 
    1070    1266.0865    -1629.887            0   -1546.2595   -18708.909 
    1080    1216.8158   -1629.6163            0   -1549.2432   -20328.557 
    1090    1186.0624   -1630.4587            0    -1552.117     -19486.9 
    1100    1196.8972   -1633.8987            0   -1554.8414   -17158.775 
    1110    1182.3764   -1635.5713            0   -1557.4731   -15120.911 
    1120    1117.0408    -1633.598            0   -1559.8153   -14963.067 
    1130    1123.3696   -1635.8801            0   -1561.6794   -16627.857 
    1140    1172.5019   -1640.6507            0   -1563.2047   -19213.399 
    1150     1202.814   -1644.0905            0   -1564.6423   -22118.061 
    1160     1124.792   -1640.2795            0   -1565.9848    -22903.13 
    1170    1101.8949   -1639.6976            0   -1566.9154   -21413.338 
    1180    1150.9717   -1643.4437            0   -1567.4198   -19045.409 
    1190    1143.7667   -

    2220     1169.102   -1692.7038            0   -1615.4824   -17528.264 
    2230    1159.8524   -1692.2524            0   -1615.6419   -16072.273 
    2240     1145.965   -1691.2868            0   -1615.5936   -15788.088 
    2250    1158.7879   -1691.8238            0   -1615.2837   -17042.588 
    2260    1202.1222   -1694.1706            0   -1614.7681   -18878.818 
    2270    1206.9371   -1693.9508            0   -1614.2303   -19226.086 
    2280    1165.1844   -1690.6676            0   -1613.7049   -18808.903 
    2290     1192.892    -1691.849            0   -1613.0562   -19132.397 
    2300    1178.4291   -1690.1628            0   -1612.3253   -19442.723 
    2310    1145.2627   -1687.1426            0   -1611.4958   -18555.516 
    2320    1180.2855   -1688.3876            0   -1610.4275   -17193.144 
    2330    1228.8438   -1690.3844            0   -1609.2169   -16691.886 
    2340    1261.5193   -1691.4487            0   -1608.1229   -16425.401 
    2350    1296.1693   -

    3420    1194.1553   -1716.7612            0    -1637.885   -14710.264 
    3430    1245.8242   -1720.0555            0   -1637.7664   -12968.122 
    3440    1266.1061   -1721.5109            0   -1637.8822   -12755.655 
    3450    1212.4605   -1718.3871            0   -1638.3018    -13652.15 
    3460    1233.4203   -1720.3143            0   -1638.8445   -16024.133 
    3470    1234.3309   -1721.0969            0    -1639.567   -18151.596 
    3480    1177.3741   -1718.1767            0   -1640.4088   -19157.522 
    3490     1187.532   -1719.6012            0   -1641.1625    -19186.77 
    3500    1194.9736   -1720.7873            0    -1641.857   -18133.691 
    3510    1180.1457   -1720.4546            0   -1642.5037   -18153.739 
    3520    1197.6959   -1722.1892            0   -1643.0791   -19374.699 
    3530    1193.2567   -1722.4416            0   -1643.6247   -19131.811 
    3540    1183.5027   -1722.3165            0   -1644.1439   -17980.182 
    3550    1141.2228   -

    4590    1189.4341   -1735.9898            0   -1657.4253   -13557.272 
    4600     1135.109   -1733.4674            0   -1658.4912   -12570.079 
    4610    1176.6027   -1736.8887            0   -1659.1718   -13016.798 
    4620    1202.6923   -1739.1356            0   -1659.6955   -13437.284 
    4630    1157.3175   -1736.6017            0   -1660.1586   -13774.801 
    4640    1157.2143   -1736.8516            0   -1660.4154   -14408.682 
    4650    1209.2949   -1740.3676            0   -1660.4913   -15131.864 
    4660    1198.2851   -1739.7346            0   -1660.5856   -14096.097 
    4670    1155.8767   -1737.0023            0   -1660.6544   -11829.825 
    4680    1196.4038   -1739.5676            0   -1660.5428   -11049.955 
    4690    1227.0544   -1741.4266            0   -1660.3773   -11240.627 
    4700      1184.31   -1738.5108            0   -1660.2848   -12183.546 
    4710    1197.5382   -1739.2261            0   -1660.1264   -13289.821 
    4720    1245.8221    

    5690    1204.1806   -1752.6479            0   -1673.1094   -14098.316 
    5700    1178.0092   -1750.6928            0    -1672.883   -13945.246 
    5710    1194.9852   -1751.5114            0   -1672.5803   -14269.246 
    5720    1222.4022   -1753.0015            0   -1672.2594   -13809.249 
    5730     1176.879   -1749.7452            0   -1672.0101   -11161.993 
    5740    1139.1359   -1746.8805            0   -1671.6384   -9652.7697 
    5750    1213.8428   -1751.1484            0   -1670.9717   -11166.779 
    5760    1223.1721   -1751.0161            0   -1670.2232   -13220.245 
    5770    1163.3423   -1746.3371            0   -1669.4961   -14649.793 
    5780    1170.9992   -1745.8812            0   -1668.5345    -15794.51 
    5790    1193.2492    -1746.068            0   -1667.2516   -16400.883 
    5800     1214.904   -1746.0398            0    -1665.793   -16130.043 
    5810    1241.6484   -1746.3857            0   -1664.3725   -15781.207 
    5820    1235.9669   -

    6890    1151.6661   -1764.6607            0   -1688.5909   -15219.301 
    6900    1116.2293   -1761.3628            0   -1687.6337   -14032.357 
    6910    1150.6108   -1762.4296            0   -1686.4295   -13426.002 
    6920       1203.4   -1764.6348            0   -1685.1479    -13429.26 
    6930    1202.5823   -1763.5226            0   -1684.0897   -14241.474 
    6940    1239.5609   -1765.1744            0    -1683.299    -16116.46 
    6950     1259.967   -1766.0707            0   -1682.8474   -17269.244 
    6960    1255.6218   -1765.6948            0   -1682.7586   -16382.834 
    6970    1270.5221   -1766.9248            0   -1683.0044   -14957.117 
    6980    1210.4042    -1763.474            0   -1683.5245   -13602.957 
    6990    1203.3002   -1763.5665            0   -1684.0861   -13151.561 
    7000    1207.1167   -1764.3663            0   -1684.6339   -11938.884 
    7010    1197.5982   -1764.2937            0     -1685.19   -9817.7369 
    7020    1173.7651   -

    7990    1147.4374   -1778.2303            0   -1702.4399   -14924.851 
    8000    1124.7493   -1776.0683            0   -1701.7764   -15213.454 
    8010    1134.1422   -1775.4532            0   -1700.5409   -14618.434 
    8020    1160.2549   -1775.4743            0   -1698.8372   -13152.207 
    8030    1180.2817   -1774.9785            0   -1697.0187   -12055.028 
    8040    1147.0971   -1771.1198            0   -1695.3518   -11641.573 
    8050     1124.152   -1768.0338            0   -1693.7814   -12249.771 
    8060    1200.6146   -1771.4813            0   -1692.1784    -13561.36 
    8070    1207.7878   -1770.5443            0   -1690.7676   -13817.867 
    8080    1173.5903    -1767.136            0   -1689.6181   -13668.062 
    8090    1221.5533   -1769.2071            0   -1688.5211   -14199.685 
    8100    1258.9132   -1770.7429            0   -1687.5892    -14006.76 
    8110    1204.8167   -1766.5514            0   -1686.9709   -12266.179 
    8120    1222.1134   -

    9090     1140.882   -1778.1033            0   -1702.7458   -14640.317 
    9100    1171.8528   -1779.3071            0    -1701.904   -13872.014 
    9110    1191.6841   -1779.6601            0   -1700.9471   -12307.413 
    9120    1201.0127    -1779.331            0   -1700.0018   -9626.5585 
    9130    1276.6762   -1783.4913            0   -1699.1643   -9303.2505 
    9140    1314.5038   -1785.5879            0   -1698.7623   -10081.585 
    9150    1223.4018   -1779.7423            0   -1698.9342    -9626.781 
    9160     1195.153   -1778.1966            0   -1699.2545   -10135.118 
    9170    1234.0006   -1781.1013            0   -1699.5931   -12119.736 
    9180    1208.6359   -1779.9249            0   -1700.0922   -13506.773 
    9190    1153.1708   -1776.8034            0   -1700.6343   -14315.472 
    9200    1149.0274   -1776.9078            0   -1701.0123   -15567.514 
    9210    1141.3617   -1776.5512            0    -1701.162   -16681.173 
    9220    1170.7972   -